In [1]:
from IPython.display import display

In [2]:
import os
import numpy as np
import pandas as pd

import json
import yaml

import re

In [3]:
import polars as pl

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import feather

In [5]:
absplice_splicemap_path = "/s/project/absplice/AbSplice_figures/data/results/_gtex_v8/junction_annotation/splicemap/"

In [6]:
absplice_splicemap_output_pq = "/s/project/rep/processed/absplice_splicemap/gtex_v8.parquet"
absplice_splicemap_output_ipc = "/s/project/rep/processed/absplice_splicemap/gtex_v8.feather"

In [7]:
absplice_subtissue_mapping = pd.read_csv("AbSplice_subtissue_mapping.csv").set_index("subtissue")["renamed_subtissue"].to_dict()
absplice_subtissue_mapping

{'Adipose_Subcutaneous': 'Adipose - Subcutaneous',
 'Adipose_Visceral_Omentum': 'Adipose - Visceral (Omentum)',
 'Adrenal_Gland': 'Adrenal Gland',
 'Artery_Aorta': 'Artery - Aorta',
 'Artery_Coronary': 'Artery - Coronary',
 'Artery_Tibial': 'Artery - Tibial',
 'Brain_Amygdala': 'Brain - Amygdala',
 'Brain_Anterior_cingulate_cortex_BA24': 'Brain - Anterior cingulate cortex (BA24)',
 'Brain_Caudate_basal_ganglia': 'Brain - Caudate (basal ganglia)',
 'Brain_Cerebellar_Hemisphere': 'Brain - Cerebellar Hemisphere',
 'Brain_Cerebellum': 'Brain - Cerebellum',
 'Brain_Cortex': 'Brain - Cortex',
 'Brain_Frontal_Cortex_BA9': 'Brain - Frontal Cortex (BA9)',
 'Brain_Hippocampus': 'Brain - Hippocampus',
 'Brain_Hypothalamus': 'Brain - Hypothalamus',
 'Brain_Nucleus_accumbens_basal_ganglia': 'Brain - Nucleus accumbens (basal ganglia)',
 'Brain_Putamen_basal_ganglia': 'Brain - Putamen (basal ganglia)',
 'Brain_Spinal_cord_cervical_c_1': 'Brain - Spinal cord (cervical c-1)',
 'Brain_Substantia_nigra':

# read splicemaps

In [8]:
splicemap_dfs = []
for file in os.listdir(absplice_splicemap_path):
    path = absplice_splicemap_path + "/" + file

    file_name_pattern = re.compile("(.+)_splicemap_(psi.).*")

    subtissue, event_type = file_name_pattern.search(file).groups()
    # # rename tissues
    # subtissue = absplice_subtissue_mapping[subtissue]

    splicemap_df = (
        pl.read_csv(path, comment_char="#")
        .lazy()
        .with_columns([
            pl.lit(subtissue).alias("tissue"),
            pl.lit(event_type).alias("event_type"),
        ])
    )
    
    splicemap_dfs.append(splicemap_df)

In [9]:
splicemap_df = pl.concat(splicemap_dfs)
del splicemap_dfs
splicemap_df.schema

{'junctions': Utf8,
 'gene_id': Utf8,
 'Chromosome': Utf8,
 'Start': Int64,
 'End': Int64,
 'Strand': Utf8,
 'splice_site': Utf8,
 'events': Utf8,
 'ref_psi': Float64,
 'k': Int64,
 'n': Int64,
 'median_n': Float64,
 'gene_name': Utf8,
 'gene_type': Utf8,
 'novel_junction': Boolean,
 'weak_site_donor': Boolean,
 'weak_site_acceptor': Boolean,
 'transcript_id': Utf8,
 'gene_tpm': Float64,
 'tissue': Utf8,
 'event_type': Utf8}

In [10]:
parsed_splicemap_df = (
    splicemap_df
    .with_columns([
        pl.col("events").str.split(";"),
        pl.col("transcript_id").str.split(";").cast(pl.List(pl.Categorical)),
    ])
    # duplicate information
    .drop("junctions")
    .sort([
        "Chromosome",
        "Start",
        "End",
        "Strand",
        "tissue",
        "gene_id",
    ])
    .collect()
)

In [11]:
del splicemap_df

In [12]:
parsed_splicemap_df = parsed_splicemap_df.lazy().with_columns([
    pl.col("events").list.eval(pl.struct([
        pl.element().str.extract('(.+):([0-9]+)-([0-9]+):(.+)', 1).alias("chrom"),
        pl.element().str.extract('(.+):([0-9]+)-([0-9]+):(.+)', 2).cast(pl.Int32).alias("start"),
        pl.element().str.extract('(.+):([0-9]+)-([0-9]+):(.+)', 3).cast(pl.Int32).alias("end"),
        pl.element().str.extract('(.+):([0-9]+)-([0-9]+):(.+)', 4).alias("strand"),
    ])).cast(pl.List(
        pl.Struct([
            pl.Field("chrom", pl.Categorical),
            pl.Field("start", pl.Int32),
            pl.Field("end", pl.Int32),
            pl.Field("strand", pl.Categorical),
        ])
    )),
    pl.struct([
        pl.col("splice_site").str.extract('(.+):([0-9]+):(.+)', 1).cast(pl.Categorical).alias("chrom"),
        pl.col("splice_site").str.extract('(.+):([0-9]+):(.+)', 2).cast(pl.Int32).alias("start"),
        pl.col("splice_site").str.extract('(.+):([0-9]+):(.+)', 3).cast(pl.Categorical).alias("strand"),
    ]).alias("splice_site"),
    *[
        pl.col(c).cast(pl.Categorical) for c in [
            "gene_id",
            "Chromosome",
            "Strand",
            "gene_name",
            "gene_type",
            "tissue",
        ]
    ],
]).collect()

In [13]:
parsed_splicemap_df.schema

{'gene_id': Categorical,
 'Chromosome': Categorical,
 'Start': Int64,
 'End': Int64,
 'Strand': Categorical,
 'splice_site': Struct([Field('chrom', Categorical), Field('start', Int32), Field('strand', Categorical)]),
 'events': List(Struct([Field('chrom', Categorical), Field('start', Int32), Field('end', Int32), Field('strand', Categorical)])),
 'ref_psi': Float64,
 'k': Int64,
 'n': Int64,
 'median_n': Float64,
 'gene_name': Categorical,
 'gene_type': Categorical,
 'novel_junction': Boolean,
 'weak_site_donor': Boolean,
 'weak_site_acceptor': Boolean,
 'transcript_id': List(Categorical),
 'gene_tpm': Float64,
 'tissue': Categorical,
 'event_type': Utf8}

In [14]:
parsed_splicemap_df.write_parquet(
    absplice_splicemap_output_pq,
    compression="snappy",
    statistics=True,
    use_pyarrow=True,
    row_group_size=1*1024*1024
)

In [17]:
df = (
    pq.read_table(
        absplice_splicemap_output_pq, 
        columns=['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'tissue', 'event_type', 'ref_psi', 'median_n', 'gene_name']
    )
    .to_pandas(strings_to_categorical=True)
)
df

,Chromosome,Start,End,Strand,gene_id,tissue,event_type,ref_psi,median_n,gene_name
0,chr1,12227,12612,+,ENSG00000223972,Testis,psi5,0.971798,8.0,DDX11L1
1,chr1,12227,12612,+,ENSG00000223972,Testis,psi3,1.000000,8.0,DDX11L1
2,chr1,12227,12612,+,ENSG00000223972,Whole Blood,psi5,1.000000,2.0,DDX11L1
3,chr1,12227,12612,+,ENSG00000223972,Whole Blood,psi3,1.000000,2.0,DDX11L1
4,chr1,12227,12645,+,ENSG00000223972,Testis,psi5,0.028202,8.0,DDX11L1
...,...,...,...,...,...,...,...,...,...,...
35365569,chrY,26621895,26622511,-,ENSG00000237917,Uterus,psi5,1.000000,1.0,PARP4P1
35365570,chrY,26621895,26622511,-,ENSG00000237917,Vagina,psi3,1.000000,3.0,PARP4P1
35365571,chrY,26621895,26622511,-,ENSG00000237917,Vagina,psi5,1.000000,3.0,PARP4P1
35365572,chrY,26621895,26622511,-,ENSG00000237917,Whole Blood,psi5,1.000000,2.0,PARP4P1


In [18]:
df.to_feather(absplice_splicemap_output_ipc)

In [20]:
absplice_splicemap_output_pq = "/s/project/rep/processed/absplice_splicemap/gtex_v8.parquet"
absplice_splicemap_output_ipc = "/s/project/rep/processed/absplice_splicemap/gtex_v8.feather"

In [21]:
!ls -lrth "/s/project/rep/processed/absplice_splicemap/"

total 1.1G
-rw-rw----+ 1 hoelzlwi ag_gagneur 780M Aug  9 21:01 gtex_v8.parquet
-rw-rw----+ 1 hoelzlwi ag_gagneur 316M Aug  9 22:27 gtex_v8.feather


In [3]:
%time
df = pd.read_feather(absplice_splicemap_output_ipc)
df = df.set_index(['Chromosome', 'Start', 'End', 'Strand'])
df

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


gene_id       tissue event_type   
Chromosome Start    End      Strand                                            
chr1       12227    12612    +       ENSG00000223972       Testis       psi5  \
                             +       ENSG00000223972       Testis       psi3   
                             +       ENSG00000223972  Whole Blood       psi5   
                             +       ENSG00000223972  Whole Blood       psi3   
                    12645    +       ENSG00000223972       Testis       psi5   
...                                              ...          ...        ...   
chrY       26621895 26622511 -       ENSG00000237917       Uterus       psi5   
                             -       ENSG00000237917       Vagina       psi3   
                             -       ENSG00000237917       Vagina       psi5   
                             -       ENSG00000237917  Whole Blood       psi5   
                             -       ENSG00000237917  Whole Blood       psi3   

                                      ref_psi  median_n gene_name  
Chromosome Start    End      Strand                                
chr1       12227    12612    +       0.971798       8.0   DDX11L1  
                             +       1.000000       8.0   DDX11L1  
                             +       1.000000       2.0   DDX11L1  
                             +       1.000000       2.0   DDX11L1  
                    12645    +       0.028202       8.0   DDX11L1  
...                                       ...       ...       ...  
chrY       26621895 26622511 -       1.000000       1.0   PARP4P1  
                             -       1.000000       3.0   PARP4P1  
                             -       1.000000       3.0   PARP4P1  
                             -       1.000000       2.0   PARP4P1  
                             -       1.000000       2.0   PARP4P1  

[35365574 rows x 6 columns]

In [4]:
%time
df.loc[("chrY", 26621895, 26622511, "-")]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


gene_id   
Chromosome Start    End      Strand                    
chrY       26621895 26622511 -       ENSG00000237917  \
                             -       ENSG00000237917   
                             -       ENSG00000237917   
                             -       ENSG00000237917   
                             -       ENSG00000237917   
...                                              ...   
                             -       ENSG00000237917   
                             -       ENSG00000237917   
                             -       ENSG00000237917   
                             -       ENSG00000237917   
                             -       ENSG00000237917   

                                                           tissue event_type   
Chromosome Start    End      Strand                                            
chrY       26621895 26622511 -             Adipose - Subcutaneous       psi5  \
                             -             Adipose - Subcutaneous       psi3   
                             -       Adipose - Visceral (Omentum)       psi3   
                             -       Adipose - Visceral (Omentum)       psi5   
                             -                      Adrenal Gland       psi5   
...                                                           ...        ...   
                             -                             Uterus       psi5   
                             -                             Vagina       psi3   
                             -                             Vagina       psi5   
                             -                        Whole Blood       psi5   
                             -                        Whole Blood       psi3   

                                     ref_psi  median_n gene_name  
Chromosome Start    End      Strand                               
chrY       26621895 26622511 -           1.0       3.0   PARP4P1  
                             -           1.0       3.0   PARP4P1  
                             -           1.0       2.0   PARP4P1  
                             -           1.0       2.0   PARP4P1  
                             -           1.0       4.0   PARP4P1  
...                                      ...       ...       ...  
                             -           1.0       1.0   PARP4P1  
                             -           1.0       3.0   PARP4P1  
                             -           1.0       3.0   PARP4P1  
                             -           1.0       2.0   PARP4P1  
                             -           1.0       2.0   PARP4P1  

[68 rows x 6 columns]

In [46]:
indexed_df = df.set_index(['Chromosome', 'Start', 'End', 'Strand'])
indexed_df

gene_id       tissue event_type   
Chromosome Start    End      Strand                                            
chr1       12227    12612    +       ENSG00000223972       Testis       psi5  \
                             +       ENSG00000223972       Testis       psi3   
                             +       ENSG00000223972  Whole Blood       psi5   
                             +       ENSG00000223972  Whole Blood       psi3   
                    12645    +       ENSG00000223972       Testis       psi5   
...                                              ...          ...        ...   
chrY       26621895 26622511 -       ENSG00000237917       Uterus       psi5   
                             -       ENSG00000237917       Vagina       psi3   
                             -       ENSG00000237917       Vagina       psi5   
                             -       ENSG00000237917  Whole Blood       psi5   
                             -       ENSG00000237917  Whole Blood       psi3   

                                      ref_psi  median_n gene_name  
Chromosome Start    End      Strand                                
chr1       12227    12612    +       0.971798       8.0   DDX11L1  
                             +       1.000000       8.0   DDX11L1  
                             +       1.000000       2.0   DDX11L1  
                             +       1.000000       2.0   DDX11L1  
                    12645    +       0.028202       8.0   DDX11L1  
...                                       ...       ...       ...  
chrY       26621895 26622511 -       1.000000       1.0   PARP4P1  
                             -       1.000000       3.0   PARP4P1  
                             -       1.000000       3.0   PARP4P1  
                             -       1.000000       2.0   PARP4P1  
                             -       1.000000       2.0   PARP4P1  

[35365574 rows x 6 columns]

In [41]:
df[['Chromosome', 'Start', 'End', 'Strand']].drop_duplicates()

,Chromosome,Start,End,Strand
0,chr1,12227,12612,+
4,chr1,12227,12645,+
5,chr1,12697,13220,+
13,chr1,12697,13224,+
14,chr1,12697,13402,+
...,...,...,...,...
35365500,chrY,26571962,26575923,+
35365501,chrY,26576014,26577423,+
35365503,chrY,26577494,26579510,+
35365504,chrY,26577494,26579588,+


In [8]:
df = pd.read_parquet(
    absplice_splicemap_output_pq,
    columns=['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'tissue', 'ref_psi', 'median_n', 'gene_name'],
    dtype_backend='pyarrow',
)
df

,Chromosome,Start,End,Strand,gene_id,tissue,ref_psi,median_n,gene_name
0,chr1,12227,12612,+,ENSG00000223972,Testis,0.971798,8.0,DDX11L1
1,chr1,12227,12612,+,ENSG00000223972,Testis,1.000000,8.0,DDX11L1
2,chr1,12227,12612,+,ENSG00000223972,Whole Blood,1.000000,2.0,DDX11L1
3,chr1,12227,12612,+,ENSG00000223972,Whole Blood,1.000000,2.0,DDX11L1
4,chr1,12227,12645,+,ENSG00000223972,Testis,0.028202,8.0,DDX11L1
...,...,...,...,...,...,...,...,...,...
35365569,chrY,26621895,26622511,-,ENSG00000237917,Uterus,1.000000,1.0,PARP4P1
35365570,chrY,26621895,26622511,-,ENSG00000237917,Vagina,1.000000,3.0,PARP4P1
35365571,chrY,26621895,26622511,-,ENSG00000237917,Vagina,1.000000,3.0,PARP4P1
35365572,chrY,26621895,26622511,-,ENSG00000237917,Whole Blood,1.000000,2.0,PARP4P1


In [34]:
df.to_feather(absplice_splicemap_output_ipc)